#### 제목 : 반도체 제조 클러스터 툴의 스케줄링 이론 및 방법 검토

1. INTRODUCTION
   - 대부분의 처리도구들은 처리량을 최대화하기 위해 웨이퍼를 카세트(25개)의 배치사이즈로 처리한다.
   - 그러나 웨이퍼 크기증가, 품질요구수준이 더 엄격해지고, 회루수축 때문에, 제조과정이 더 복잡해지고있다.
   - 그래서 채택한게 SWP(Single-Wafer Processing) : 웨이퍼를 챔버 안에서 하나하나씩 처리하는 기술
   - 그런데 SWP단점이 챔버간 이송작업이 너무많다.
   - 그래서 여러 챔버들을 하나의 tool로 결합하고자 한다.
   - 그것이 바로 Cluster tools(track equipment) : wafer handling robot
   <br><br>
   - 이 논문에서 소개할 내용
   - 1. tool architecture
   - 2. operation issues
   - 3. scheduling requirements
   - 4. recent progress in tool science
   - 5. engineering for scheduling
   - 6. control of cluster tools
   - 그 외에도 schedule quality의 개념, minimizing the tool cycle time, controlling wafer delays, workload balancing의 개념, dispatching rules for cluster tool scheduling등을 소개하면서
   - 최종적으로, 스케줄링과 제어를 위한 제어 소프트웨어 아키텍쳐를 설명

2. CLUSTER TOOL ARCHITECTURES, OPERATION, AND SCHEDULING REQUIREMENTS
    <br>
    <br>
    - 2.1. Basic Cluster Tool Architecture and Operation
        - 보통 cluster tool은 방사형 모양이고, 6개 정도의 챔버, single wafer-handling robot 1개를 가진다.
        - 챔버가 추가되거나 빠질수도 있는데, 그러면 길고 non-uniform한 이동시간이 유발된다. 로봇 또한 팔을 하나 가질수도, 두 개(dual-arm)를 가질수도 있다. 단, 팔이 두 개일 때는, 움직이는 공간을 반씩 나눠 움직이는 각도를 고정해놓는다.
        - Dual-armed tool은 더 큰 처리량을 보이지만 그만큼 비용도 커진다.
        - 보통 cassette는 loadlock에 저장됨, 앞으로 편의를 위해, loadlock에 저장된 cassette를 그냥 loadlock이라 하자.
        #### **질문 : loadlock은 그냥 처리중인 생산품을 저장해놓는 역할?**
        - 카세트에 적재된 웨이퍼 loadlock에서 unload -> 챔버1로 load -> 다 되면 unload -> 챔버2로 load -> 그러면서 새 웨이퍼 챔버1로 load -> 하나하나씩 반복
        - 처리시간이 긴 처리단계는 2개 이상의 병렬의 챔버가 필요할수도 있다.
        - 따라서 챔버를 통과하는 일반적인 웨이퍼 흐름 패턴은 직렬 병렬이다.
        - cassette에 담긴 마지막 웨이퍼가 챔버에 실리면, 그 옆의 loadlock에 load된 wafer를 꺼내서 쓴다. 그러면서 계속 loadlock에 cassette를 계속 보충해준다.
        #### **질문 : 방금 위에 적은 내용 제대로 이해한거 맞나??**
        - aligner : 웨이퍼가 arm에 잘 위치할수있게 정렬해주는 역할(레이저 빔으로 정렬해줌)
        - 기계적인 문제로 웨이퍼가 loadlock에서 꺼내질때, arm에 잘 위치하지 못할수도 있으니까 aligner가 챔버의 슬롯에 잘 들어갈 수 있도록 위치를 재정렬해줌
        - align 소요시간 : 5~10초
        - 쿨러 모듈도 있을수 있음(웨이퍼가 뜨거우면 loadlock에 있는 다른 웨이퍼에도 영향을 줄 수 있기 떄문)
        - 공간 제약으로 인해 챔버간 버퍼는 없음
        - 간혹 있는 경우도 있는데 이러면 로봇 작업량 증가, 웨이퍼가 과도하게 냉각됨
    <br>
    <br>
    - 2.2 Advanced Cluster Tool Architectures and Operation
        - 챔버는 load된 웨이퍼를 처리하기위해 진공, 환기 사이클을 반복한다.
        - 몇몇 tool들은 진공버퍼를 챔버와 loadlock사이에 둬서 챔버의 전체영역을 진공상태로 유지한다. 진공, 환기사이클을 만들지 않기 위해.
        - 몇몇 최신 tool들은 챔버에 다중 웨이퍼슬롯을 가진다. 이것은 여러개의 웨이퍼를 함께 처리하기 때문에 SWP보다 생산량을 더 증가시킨다. 그러나 그만큼 스케줄 복잡도도 상당히 증가한다.
        - 다중 핸들링 로봇과 SWP 챔버의 통합 시스템을 track tool 또는 track system이라고 한다.
        #### **질문 : 포토리소그래피 공정은 광민성 화학 물질로 코팅된 웨이퍼를 스테퍼에 공급하고 스테퍼에서 회로 패턴 사진 이미지에 노출되어 형성된 웨이퍼의 회로 패턴을 개발하는 트랙 시스템을 사용한다.???**
        - 포토리소그래피 공정, Process module, 자동 습식 스테이션 도 여러개의 module, robot들로 시스템이 구성되어있다.
        - 최근에는 웨이퍼의 테스트디바이스를 위한 EDS프로세서가 track system의 형식으로 구성되어있다.
        - rail-guided vehicle(RGV)로 불리는 레일에서 움직이는 로봇이 load,unload작업을 하고 다중 병렬로 연결된 여러 테스트 툴들이 있다.
        - RGV의 service zone(움직이는 구간)은 서로 조금씩 곂쳐져있고 공정 중간의 버퍼가 없어서 로봇 충돌이나 deadlock이 발생할 수도 있다.
        - 몇몇 cluster tool은 딜레이를 줄이기위해 중간버퍼를 연결해 작업 끝난 웨이퍼를 바로 다음 tool로 넘기는 것도 있다.
        - radial-tpye의 cluster tool들은 오직 로테이션하게 움직이기도 하고, wet station의 경우 수평적으로 움직이기만 하고, track equipment는 수직,수평적으로 움직이기도 한다.
        - 이렇게 각각 다른 로봇 움직임 패턴은 각기 다른 스케줄링 결과를 만들어낸다.
    <br>
    <br>
    - 2.3 Tool Scheduling Requirements
    
